In [1]:
import json
import os

from pydantic import BaseModel, Field
from config import experiment_settings as settings

from src.document_ingestion import DocumentIngestionPipeline
from src.embedder import EmbedderModule
from src.models import PipelineSettings, InputFile

# os.environ["HF_HOME"] = settings.docling_model_weight_dir
# os.environ["DOCLING_ARTIFACTS_PATH"] = settings.docling_model_weight_dir
os.environ["DOCLING_ARTIFACTS_PATH"] = os.path.join(
    settings.docling_model_weight_dir, "docling-models"
)
# os.environ["DOCLING_ARTIFACTS_PATH"] = str(os.path.join(settings.docling_model_weight_dir, "artifacts"))
# os.environ["DOCLING_ARTIFACTS_PATH"] = "/Users/id4thomas/models/document_parsing/docling/hub/models--ds4sd--docling-models/snapshots/36bebf56681740529abd09f5473a93a69373fbf0"

/opt/miniconda3/envs/psiking/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
file_dir = os.path.join(
    settings.data_dir, "retrieval_dataset/2503-01-korean-finance/kr-fsc_policy"
)

fnames = [x for x in os.listdir(file_dir) if "pdf" in x][:10]
fnames

['83566_1.pdf',
 '82566_1.pdf',
 '79122_2.pdf',
 '79535_4.pdf',
 '81364_1.pdf',
 '80078_4.pdf',
 '81307_2.pdf',
 '82814_1.pdf',
 '82390_7.pdf',
 '82787_1.pdf']

In [3]:
metadata_file_path =  os.path.join(
    settings.data_dir, "retrieval_dataset/2503-01-korean-finance/kr-fsc_pdf_file_metadata.json"
)
with open(metadata_file_path, "r") as f:
    metadata_file_contents = json.load(f)

In [4]:
metadata_file_contents[0]

{'page_no': 1,
 'item_no': '2718',
 'item_id': '84155',
 'item_date': '2025-03-13',
 'item_title': '[보도참고] 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 약 50bp 절감합니다 - 「신용보증기금법」 개정안, 국회 본회의 통과',
 'no': 1,
 'name': '250313(보도참고) 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 약 50bp 절감합니다.pdf',
 'extension': 'pdf'}

In [5]:
metadata_dicts = {
    "{}_{}.pdf".format(x["item_id"], x["no"]): {
        "title": x["item_title"]
    }
    for x in metadata_file_contents
}

In [6]:
metadata_dicts["84155_1.pdf"]

{'title': '[보도참고] 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 약 50bp 절감합니다 - 「신용보증기금법」 개정안, 국회 본회의 통과'}

In [7]:
## Prepare Input Files
input_files = []
for fname in fnames[:2]:
    file_path = str(os.path.join(file_dir, fname))
    
    input_file = InputFile(
        file_path=file_path,
        metadata=metadata_dicts[fname]
    )
    input_files.append(input_file)

In [8]:
input_files[0]

InputFile(uid='2c516000-d3ae-486a-a305-71e47aad46c3', file_path='/Users/id4thomas/datasets/retrieval_dataset/2503-01-korean-finance/kr-fsc_policy/83566_1.pdf', metadata={'title': '[보도참고] 김병환 금융위원장, 외국계 금융회사 간담회 주재'})

# 1. IngestionPipeline

## 1-1. Initialize

In [9]:
pipeline_settings = PipelineSettings(
    vlm_base_url = settings.vlm_base_url,
    vlm_api_key=settings.vlm_api_key,
    vlm_model = settings.vlm_model,
    poppler_path="/opt/homebrew/Cellar/poppler/25.01.0/bin",
    text_chunk_size=1024,
    text_chunk_overlap=128
)

In [10]:
pipeline = DocumentIngestionPipeline(pipeline_settings)

## 1-2. Run

In [11]:
documents = pipeline.run(input_files)

0it [00:00, ?it/s]

NUM IMAGES TO ANNOTATE 2


1it [00:04,  4.27s/it]

NUM IMAGES TO ANNOTATE 5


2it [00:15,  7.72s/it]


In [12]:
len(documents)

16

In [13]:
documents[:2]

[Document(id_='9c483e3a-cb5d-44f5-a7e2-6ab8fe428b79', metadata={'title': '[보도참고] 김병환 금융위원장, 외국계 금융회사 간담회 주재', 'source_id': '2c516000-d3ae-486a-a305-71e47aad46c3', 'source_file': '83566_1.pdf'}, nodes=[TextNode(id_='bf93a09c-507a-48be-b793-8381dae1c449', metadata={'page_no': 1}, text_type=<TextType.PLAIN: 'plain'>, label=<TextLabel.PLAIN: 'plain'>, resource=MediaResource(data=None, text="보도참고자료\n광부\n보도시점\n배포 시\n배포 2024.12.10.(화) 15:30\n김병환 금융위원장, 외국계 금융회사 간담회 주재\n'24.12.10(화)  금융위원회는  김병환  금융위원장 주재로 외국계 금융회사 대표들이 참여하는 간담회를 개최하고, 한국의 정치경제 상황에 대한 해외 ․ 시각을 점검하는 한편, 한국 정부가 금융시장 안정을 유지할 수 있는 충분한 역량과 의지를 가지고 있음을 설명하였다.\n- ◇ 일시/장소 : '24.12.10(화) 15:00, 금융위원회 대회의실\n- ◇ 참석기관: S&P, 무디스, Citi은행, HSBC은행, 도이치방크, JP모건체이스, 뱅크오브아메리카 소시에테제네랄,  미쓰이스미토모  은행,  CCB,  골드만삭스,  맥쿼리,  모건스탠리, JP모건 증권, BNP파리바, 나틱시스증권, SC증권, ING증권, IMC증권, HSBC증권\n이  날  회의에서 김병환 금융위원장은 정치적 불확실성이 커진 상황이나, 경제  문제만큼은  경제부총리  등  경제팀을  중심으로  일관되고  안정적으로 관리해 나가고 있다고 강조하였다. 김 위원장은 시장 안정을 위한 정부의 준비태세는 확고히 유지되고 있으며, 부동산 PF 연착륙, 기업 밸류업, 자본 시

In [14]:
documents[0].metadata

{'title': '[보도참고] 김병환 금융위원장, 외국계 금융회사 간담회 주재',
 'source_id': '2c516000-d3ae-486a-a305-71e47aad46c3',
 'source_file': '83566_1.pdf'}

In [15]:
print(documents[0].nodes[0].text)

보도참고자료
광부
보도시점
배포 시
배포 2024.12.10.(화) 15:30
김병환 금융위원장, 외국계 금융회사 간담회 주재
'24.12.10(화)  금융위원회는  김병환  금융위원장 주재로 외국계 금융회사 대표들이 참여하는 간담회를 개최하고, 한국의 정치경제 상황에 대한 해외 ․ 시각을 점검하는 한편, 한국 정부가 금융시장 안정을 유지할 수 있는 충분한 역량과 의지를 가지고 있음을 설명하였다.
- ◇ 일시/장소 : '24.12.10(화) 15:00, 금융위원회 대회의실
- ◇ 참석기관: S&P, 무디스, Citi은행, HSBC은행, 도이치방크, JP모건체이스, 뱅크오브아메리카 소시에테제네랄,  미쓰이스미토모  은행,  CCB,  골드만삭스,  맥쿼리,  모건스탠리, JP모건 증권, BNP파리바, 나틱시스증권, SC증권, ING증권, IMC증권, HSBC증권
이  날  회의에서 김병환 금융위원장은 정치적 불확실성이 커진 상황이나, 경제  문제만큼은  경제부총리  등  경제팀을  중심으로  일관되고  안정적으로 관리해 나가고 있다고 강조하였다. 김 위원장은 시장 안정을 위한 정부의 준비태세는 확고히 유지되고 있으며, 부동산 PF 연착륙, 기업 밸류업, 자본 시장  선진화  등  주요  정책  과제들도  계획된  일정대로  차질  없이  추진해 나갈 것이라고 언급하면서, 앞으로 외국계금융회사와의 더욱 긴밀한 소통을 통해 한국의 상황과 정부의 계획에 대한 충분한 설명이 이루어질 수 있도록 하겠다고 밝혔다.
이  날  회의에서  외국계  금융회사  참석자들은  이번  사태가  한국  경제의 근간을 훼손하거나 지속적인 악영향을 미칠 것으로 생각하지 않으며, 정치적 불확실성이 조속히 해소된다면 시장도 빠르게 안정을 찾을 수 있을 것이라고 전망하였다. 다만 주식시장 등의 단기 변동성을 완화하기 위해 연기금 등 기관투자자들의 보다 적극적인 역할이 필요하다는 의견도 제시되었다.


In [16]:
# print(documents[3].nodes[0].text)

In [17]:
documents[6].nodes

[TextNode(id_='0256b83d-579c-40d1-9eb0-ab967f413e18', metadata={'page_no': 1}, text_type=<TextType.PLAIN: 'plain'>, label=<TextLabel.PLAIN: 'plain'>, resource=MediaResource(data=None, text="< 5.14일 발표한 부동산PF 연착륙 대책 中 대주단 협약 개정 관련 내용 >\n- □  사업장의 만기연장이자유예 조건 ‧ 을 강화 하는  방향으로 협약을 개정 하여 무분별한 만기연장이자유예 ‧ 제한\n- ⅰ) 2회 이상 만기 연장 시 외부전문기관의 PF 사업성평가 를 의무화 하고, 만기연장 동의 기준 조정 (2/3  이상 → 3/4 이상)\n- ⅱ)  협약을 통한 이자유예  기존 연체이자 는 의 상환 * 을 전제 로 추진하도록 의무화하고, 연체이자를 고려한 건전성분류 적용\n- *  ①기존  연체이자의 전액 상환을 전제로 이자유예가 원칙이나, ②연체이자 일정부분 상환 시 잔여연체해소계획을 고려하여 유예 가능\n- ⅲ)  대주단의 만기연장이자유예 내용 ‧ 을 사무국에 의무적으로 통보 하도록 하여 PF사업장 재구조화정리 상황 상시 모니터링 ‧\n<PF 상설협의회 구성 등>\n▸ (구    성)  협회 중앙회  : ‧ 전국은행연합회,  농협중앙회,  수협중앙회,  산림조합중앙회, 신협중앙회, 새마을금고중앙회, 금융투자협회, 생명보험협회, 손해보험협회, 여신금융협회, 저축은행중앙회의 대표자 관계기관 : 신용보증기금, 기술보증기금, 한국주택금융공사, 한국무역보험공사, 주택도시보증공사, 한국자산관리공사, 연합자산관리주식회사의 대표자 ▸ (업  무) 협약의 제개정 ‧ 등에 관한 사항을 심의의결 ‧\n1 PF  대주단 협약 운영 상황\n'23.4.24일  全  금융권 ｢ PF  대주단  협약 이  개정·시행된  이후  채권금융 ｣ 기관들은 협약상 제도 및 절차 등에 따라 PF 사업장의 원활한 정상화와 재구조화를 추진해 왔습니다. 이와 함

In [18]:
from psiking.core.base.schema import TextNode, ImageNode, TableNode
def count_node(documents, node_cls = TextNode):
    counts = [
        len(list(filter(lambda x: isinstance(x, node_cls), x.nodes)))
        for x in documents
    ]
    return sum(counts)

print("TextNode: {}".format(count_node(documents, node_cls=TextNode)))
print("ImageNode: {}".format(count_node(documents, node_cls=ImageNode)))
print("TableNode: {}".format(count_node(documents, node_cls=TableNode)))

TextNode: 7
ImageNode: 5
TableNode: 4


## 1-3. Save to DocumentStore

In [19]:
from psiking.core.storage.docstore import InMemoryDocumentStore

docstore = InMemoryDocumentStore()

In [20]:
docstore.add(documents)
print(docstore.count())

16


# 2. Embedding Pipeline

In [21]:
class EmbeddingSettings(BaseModel):
    # PictureDescription
    embedding_model_path: str = Field("")
    
    dense_batch_size: int = Field(4)
    sparse_batch_size: int = Field(256)

In [22]:
embedding_settings = EmbeddingSettings(
    embedding_model_path=str(os.path.join(settings.model_weight_dir, "embedding")),
    dense_batch_size=4,
    sparse_batch_size=256
)


In [23]:
embedder = EmbedderModule(embedding_settings)

Loaded Dense Embedder
Loaded Sparse Embedder


In [24]:
embeddings = embedder.run(documents)

100%|██████████| 2/2 [00:03<00:00,  1.72s/it]


# 3. VectorStore

In [25]:
from qdrant_client import QdrantClient
from psiking.core.storage.vectorstore.qdrant import QdrantSingleHybridVectorStore


# initialize client
client = QdrantClient(":memory:")
collection_name = "kr-fsc_policy"

vector_store = QdrantSingleHybridVectorStore(
    collection_name=collection_name,
    client=client
)

In [26]:
## Create Collection
from qdrant_client.http import models

# bge-m3 1024 dim
dense_embedding_dim=1024
dense_vectors_config = models.VectorParams(
    size=dense_embedding_dim,
    distance=models.Distance.COSINE,
    on_disk=True,
    hnsw_config = {
        "m": 16,
        "ef_construct": 100,
    }
)

# Sparse BM42 Embedding
sparse_vectors_config = models.SparseVectorParams(
    modifier=models.Modifier.IDF, ## uses indices from bm42 embedder
)

# Create VectorStore
vector_store.create_collection(
    dense_vector_config=dense_vectors_config,
    sparse_vector_config=sparse_vectors_config,
    on_disk_payload=True,
)

In [27]:
vector_store.add(
    documents=documents,
    texts=embeddings.texts,
    dense_embeddings=embeddings.dense.values,
    sparse_embedding_values=embeddings.sparse.values,
    sparse_embedding_indices=embeddings.sparse.indices,
    metadata_keys=["source_file", "title"]
)

# Retrieval Test

In [28]:
from psiking.core.base.schema import TextNode, ImageNode, TableNode, Document

query="정치적 불확실성이 한국 경제에 미치는 영향"
query_document = Document(
    nodes=[TextNode(text=query)]
)

In [29]:
query_embedding_output = embedder.run([query_document])

100%|██████████| 1/1 [00:00<00:00,  5.04it/s]


In [30]:
# Hybrid Query
results = vector_store.query(
    mode="hybrid",
    dense_embedding=query_embedding_output.dense.values[0],
    sparse_embedding_values=query_embedding_output.sparse.values[0],
    sparse_embedding_indices=query_embedding_output.sparse.indices[0],
    limit=10
)
print(len(results.points))

10


In [31]:
for point in results.points[:5]:
    point_id = point.id
    point_chunk = docstore.get([point_id])[0]
    print("{} - score {:.3f}".format(point_id, point.score))
    print(point_chunk.metadata)
    print(type(point_chunk.nodes[0]))
    print(repr(point_chunk.nodes[0].text[:100]))
    print('-'*30)

9c483e3a-cb5d-44f5-a7e2-6ab8fe428b79 - score 1.000
{'title': '[보도참고] 김병환 금융위원장, 외국계 금융회사 간담회 주재', 'source_id': '2c516000-d3ae-486a-a305-71e47aad46c3', 'source_file': '83566_1.pdf'}
<class 'psiking.core.base.schema.TextNode'>
"보도참고자료\n광부\n보도시점\n배포 시\n배포 2024.12.10.(화) 15:30\n김병환 금융위원장, 외국계 금융회사 간담회 주재\n'24.12.10(화)  금융위원회는  김병환  금융"
------------------------------
62824d61-999e-4045-81ac-bec31363434f - score 0.333
{'title': '[보도자료] PF 연착륙을 위한 全 금융권 ｢대주단 협약｣ 개정', 'source_id': '3995c297-bb75-4314-91f1-af77b8faf811', 'source_file': '82566_1.pdf'}
<class 'psiking.core.base.schema.TextNode'>
'- *  PF대주단협의회의  원활한  운영을  위해  전국은행연합회에 설치되었으며 자료 수집, 기록 및 문서관리 등 협약 운영에 필요한 지원업무를 수행\n3 기대효과 및 향후 계획\n'
------------------------------
4ce78572-9fb8-4509-93ea-5fd3ca55dbf9 - score 0.250
{'title': '[보도자료] PF 연착륙을 위한 全 금융권 ｢대주단 협약｣ 개정', 'source_id': '3995c297-bb75-4314-91f1-af77b8faf811', 'source_file': '82566_1.pdf'}
<class 'psiking.core.base.schema.TextNode'>
'2 주요 개정 내용\n사업성이  극히  낮은  사업장에  대해서도  반복적으로 